In [1]:
# импортируем необходимые библиотеки и классы
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from statsmodels.formula.api import logit
np.set_printoptions(precision=None, suppress=True)

In [2]:
# загружаем и смотрим данные 
data = pd.read_csv('Data/Bankloan.csv', sep=';', decimal=',')
data.head(5)

,age,job,debtinc,creddebt,othdebt,default
0,28,working - other,17.7,2.990592,4.797408,0
1,64,working - production,14.7,5.047392,12.004608,0
2,40,working - IT,4.8,1.042368,1.885632,0
3,30,working - IT,34.5,1.751220,7.563780,0
4,25,working - IT,22.4,0.759360,5.960640,1


In [3]:
# формируем массив меток и массив предикторов
y = data.pop('default')
# создаем список количественных предикторов
num_columns = data.select_dtypes(exclude='object').columns
# создаем экземпляр класса StandardScaler
scal = StandardScaler()
# выполняем стандартизацию
data[num_columns] = scal.fit_transform(data[num_columns])
# выполняем дамми-кодирование
X = pd.get_dummies(data, drop_first=True)
# записываем названия предикторов
feat_labels = X.columns

In [4]:
# создаем экземпляр класса LogisticRegression, 
# отключаем регуляризации
logreg_sklearn = LogisticRegression(solver='lbfgs', 
                                    penalty='none')
# обучаем модель
logreg_sklearn.fit(X, y);

Найдем стандартные ошибки коэффициентов модели, которые вычисляются как    
квадратный корень из диагональных элементов матрицы ковариации предикторов. <br>

Матрица ковариации может быть получена следующим образом:
(X.T * V * X) ** (-1), где  
> X - матрица признаков с добавленным в начало столбцом из единиц (для константы)  
> V - диагональная матрица, c диагональными элементами v[i, i] =  pi * (1 - pi),  
> где pi - предсказанная вероятность положительного класса для i-го наблюдения 

In [5]:
# вычисляем матрицу спрогнозированных вероятностей классов
predProbs = logreg_sklearn.predict_proba(X)
print(predProbs.shape)
print('')
print(np.round(predProbs, 3))

(1500, 2)

[[0.289 0.711]
 [0.914 0.086]
 [0.921 0.079]
 ...
 [0.722 0.278]
 [0.687 0.313]
 [0.773 0.227]]


In [6]:
# создаем матрицу плана, добавляем 
# в начало столбец с единицами 
X_ = np.hstack([np.ones((X.shape[0], 1)), X])
print(X_.shape)
print('')
print(np.round(X_, 3))

(1500, 9)

[[ 1.    -0.47   1.165 ...  0.     1.     0.   ]
 [ 1.     2.27   0.715 ...  0.     0.     1.   ]
 [ 1.     0.443 -0.769 ...  1.     0.     0.   ]
 ...
 [ 1.    -0.698 -1.009 ...  0.     0.     1.   ]
 [ 1.    -0.013  0.685 ...  1.     0.     0.   ]
 [ 1.    -0.546 -0.619 ...  1.     0.     0.   ]]


In [7]:
# создаем диагональную матрицу, помещаем на главную 
# диагональ произведение вероятностей отрицательного и
# положительного классов для каждого наблюдения
V = np.diagflat(np.product(predProbs, axis=1))
print(V.shape)
print("")
print(V)

(1500, 1500)

[[0.20530004 0.         0.         ... 0.         0.         0.        ]
 [0.         0.07837879 0.         ... 0.         0.         0.        ]
 [0.         0.         0.07273397 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.20090302 0.         0.        ]
 [0.         0.         0.         ... 0.         0.21486664 0.        ]
 [0.         0.         0.         ... 0.         0.         0.17549169]]


In [8]:
# теперь вычисляем и выводим ковариационную матрицу

'''
Обратите внимание, что в Python 3.5+ оператор @ выполняет 
умножение матриц, таким образом, если вы используете 
Python 3.5+, вы можете заменить строчку кода ниже
на более читаемый вариант:
cov_matrix = np.linalg.pinv(X_.T @ V @ X_)
'''
cov_matrix = np.linalg.inv(np.dot(np.dot(X_.T, V), X_))
print("Ковариационная матрица:\n", np.round(cov_matrix, 4))

Ковариационная матрица:
 [[ 0.0185  0.0022  0.0008 -0.     -0.0018 -0.018  -0.0186 -0.0177 -0.0185]
 [ 0.0022  0.0104  0.0001 -0.0041 -0.0018  0.0008 -0.0011  0.0019 -0.0022]
 [ 0.0008  0.0001  0.0069 -0.0006 -0.0033 -0.0016 -0.0029 -0.0014 -0.0006]
 [-0.     -0.0041 -0.0006  0.0133 -0.0065 -0.001  -0.     -0.0013 -0.001 ]
 [-0.0018 -0.0018 -0.0033 -0.0065  0.0114  0.0021  0.003   0.0018  0.0017]
 [-0.018   0.0008 -0.0016 -0.001   0.0021  0.0364  0.0187  0.0186  0.0179]
 [-0.0186 -0.0011 -0.0029 -0.      0.003   0.0187  0.048   0.0185  0.0184]
 [-0.0177  0.0019 -0.0014 -0.0013  0.0018  0.0186  0.0185  0.0299  0.0176]
 [-0.0185 -0.0022 -0.0006 -0.001   0.0017  0.0179  0.0184  0.0176  0.0912]]


In [9]:
# вычисляем коэффициенты, z-значения, статистики Вальда, p-значения
coefs = np.round(np.insert(
    logreg_sklearn.coef_, 0, logreg_sklearn.intercept_), 4)
se = np.round(np.sqrt(np.diag(cov_matrix)), 4)
z_values = np.round(coefs / se, 4)
wald_values = np.round((coefs / se) ** 2, 4)
p_values = np.round((1 - stats.norm.cdf(abs(z_values))) * 2, 4)

In [10]:
# сохраняем результаты вычислений в датафрейм
feat_labels = np.insert(feat_labels, 0, 'Intercept')
summary = np.array([coefs, se, z_values, wald_values, p_values])
columns = ['coef', 'se', 'z_value', 'Wald', 'p_value']
logit_skl_summary = pd.DataFrame(summary.T, 
                                 index=feat_labels,
                                 columns=columns)
logit_skl_summary

,coef,se,z_value,Wald,p_value
Intercept,-0.3205,0.1362,-2.3532,5.5373,0.0186
age,-1.1907,0.1019,-11.6850,136.5389,0.0000
debtinc,0.8541,0.0833,10.2533,105.1302,0.0000
creddebt,0.8140,0.1155,7.0476,49.6689,0.0000
othdebt,-0.2967,0.1070,-2.7729,7.6890,0.0056
job_own business,-0.5412,0.1908,-2.8365,8.0456,0.0046
job_working - IT,-0.8158,0.2191,-3.7234,13.8638,0.0002
job_working - other,-0.5678,0.1729,-3.2840,10.7845,0.0010
job_working - production,-0.3522,0.3020,-1.1662,1.3601,0.2435


In [11]:
# конкатенируем массив признаков и массив меток
data = pd.concat([X, y], axis=1)
# преобразовываем имена столбцов в формат, понятный для statmodels
data.rename(columns=lambda x: x.replace(' ', '_').replace('-', '_'), 
            inplace=True)
# создаем и выводим формулу
columns = data.columns.drop('default')
formula = 'default ~ '
for col in columns:
    formula = formula + str(col) + ' + '
formula

'default ~ age + debtinc + creddebt + othdebt + job_own_business + job_working___IT + job_working___other + job_working___production + '

In [12]:
# редактируем формулу, удалив три лишних символа с конца
formula = formula[0: -3]
formula

'default ~ age + debtinc + creddebt + othdebt + job_own_business + job_working___IT + job_working___other + job_working___production'

In [13]:
# создаем экземпляр класса logit и обучаем 
logreg_statsmodels = logit(formula, data).fit()

Optimization terminated successfully.
         Current function value: 0.500446
         Iterations 6


In [14]:
# смотрим полученные результаты
print(logreg_statsmodels.summary())

                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                 1500
Model:                          Logit   Df Residuals:                     1491
Method:                           MLE   Df Model:                            8
Date:                Sat, 11 Feb 2023   Pseudo R-squ.:                  0.2376
Time:                        15:48:08   Log-Likelihood:                -750.67
converged:                       True   LL-Null:                       -984.64
Covariance Type:            nonrobust   LLR p-value:                 5.294e-96
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.3205      0.136     -2.354      0.019      -0.587      -0.054
age                         -1.1907      0.102    -11.680      0.000      -1.391      -0.